# Implementation of is_seperable() function
## Step 1: Determine if the state is PSD

In [1]:
using LinearAlgebra
using Ket
using TensorCore

In [2]:
tol = eps()^(3/4)
tol

1.8189894035458565e-12

In [3]:
function has_positive_eigenvals(X)
    for i in eigvals(X)
        if !(i >= tol)
            print(i)
            return false
        end
    end
    return true
end

has_positive_eigenvals (generic function with 1 method)

In [4]:
# Source https://math.stackexchange.com/questions/357980/how-to-generate-random-symmetric-positive-definite-matrices-using-matlab
function generate_random_symmetric_psd(d)
    A = rand(Float64,(d,d))
    A = 0.5*(A+A')
    A = A + d * I
    A = A / tr(A)
    return A
end

generate_random_symmetric_psd (generic function with 1 method)

In [5]:
d = 4
A = kron(generate_random_symmetric_psd(d),generate_random_symmetric_psd(d))
B = generate_random_symmetric_psd(d)


4×4 Matrix{Float64}:
 0.274253   0.0201374    0.0477959    0.0260544
 0.0201374  0.236307     0.000407752  0.015734
 0.0477959  0.000407752  0.227435     0.0367458
 0.0260544  0.015734     0.0367458    0.262006

In [6]:
println(isposdef(A))
println(isposdef(B))


true
true


In [7]:
function is_ppt(X)
    # lX = length(X); NO!! This is matlab notation
    lX = maximum(size(X))

    dim = round(Int, sqrt(lX))
    println(dim)
    return isposdef(Ket.partial_transpose(X,2))
end

is_ppt (generic function with 1 method)

In [8]:
function is_seperable(X)
    seperable = -1

    lX = maximum(size(X))
    rX = rank(X);
    X = X/tr(X);
    sep = -1;
    dim = round(Int, sqrt(lX))


    nD = min(dim)
    xD = max(dim)
    pD = prod(dim)
    
    println("------")
    println(dim)
    println(nD)
    println(xD)
    println(pD)
    println("-------")

    if(nD == 1)
        println("Every positive semidefinite matrix is separable when one of the local dimensions is 1");
        return 1       
    end
    
    XA = Ket.partial_trace(X,[2]);
    XB = Ket.partial_trace(X,[1]);

    # pre-load various references
    # Définir un dictionnaire pour stocker les références
    refs = Dict(
        1 => "A. Peres. Separability criterion for density matrices. Phys. Rev. Lett., 77:1413-1415, 1996.",
        2 => "M. Horodecki, P. Horodecki, and R. Horodecki. Separability of mixed states: Necessary and sufficient conditions. Phys. Lett. A, 223:1-8, 1996.",
        3 => "P. Horodecki, M. Lewenstein, G. Vidal, and I. Cirac. Operational criterion and constructive checks for the separability of low-rank density matrices. Phys. Rev. A, 62:032310, 2000.",
        4 => "K. Chen and L.-A. Wu. A matrix realignment method for recognizing entanglement. Quantum Inf. Comput., 3:193-202, 2003.",
        5 => "F. Verstraete, J. Dehaene, and B. De Moor. Normal forms and entanglement measures for multipartite quantum states. Phys. Rev. A, 68:012103, 2003.",
        6 => "K.-C. Ha and S.-H. Kye. Entanglement witnesses arising from exposed positive linear maps. Open Systems & Information Dynamics, 18:323-337, 2011.",
        7 => "O. Gittsovich, O. Guehne, P. Hyllus, and J. Eisert. Unifying several separability conditions using the covariance matrix criterion. Phys. Rev. A, 78:052319, 2008.",
        8 => "L. Gurvits and H. Barnum. Largest separable balls around the maximally mixed bipartite quantum state. Phys. Rev. A, 66:062311, 2002.",
        9 => "H.-P. Breuer. Optimal entanglement criterion for mixed quantum states. Phys. Rev. Lett., 97:080501, 2006.",
        10 => "W. Hall. Constructions of indecomposable positive maps based on a new criterion for indecomposability. E-print: arXiv:quant-ph/0607035, 2006.",
        11 => "A. C. Doherty, P. A. Parrilo, and F. M. Spedalieri. A complete family of separability criteria. Phys. Rev. A, 69:022308, 2004.",
        12 => "M. Navascues, M. Owari, and M. B. Plenio. A complete criterion for separability detection. Phys. Rev. Lett., 103:160404, 2009.",
        13 => "N. Johnston. Separability from spectrum for qubit-qudit states. Phys. Rev. A, 88:062330, 2013.",
        14 => "C.-J. Zhang, Y.-S. Zhang, S. Zhang, and G.-C. Guo. Entanglement detection beyond the cross-norm or realignment criterion. Phys. Rev. A, 77:060301(R), 2008.",
        15 => "R. Hildebrand. Semidefinite descriptions of low-dimensional separable matrix cones. Linear Algebra Appl., 429:901-932, 2008.",
        16 => "R. Hildebrand. Comparison of the PPT cone and the separable cone for 2-by-n systems. http://www-ljk.imag.fr/membres/Roland.Hildebrand/coreMPseminar2005_slides.pdf",
        17 => "D. Cariello. Separability for weak irreducible matrices. E-print: arXiv:1311.7275 [quant-ph], 2013.",
        18 => "L. Chen and D. Z. Djokovic. Separability problem for multipartite states of rank at most four. J. Phys. A: Math. Theor., 46:275304, 2013.",
        19 => "G. Vidal and R. Tarrach. Robustness of entanglement. Phys. Rev. A, 59:141-155, 1999."
    )


    if(!isposdef(X))
        error("Not positive semidefinite, can't determine separability")
    end

    ppt = is_ppt(X)
    if(!ppt)
        print("Determined to be entangled via the PPT criterion. Reference:\n",refs[1],"\n")
        return 0

    elseif(pD <= 6 || min(dim) <= 1)
        println("Determined to be separable via sufficiency of the PPT criterion in small dimensions. Reference:\n",refs[2],"\n")
        return 1
    end

    return seperable
    
end

is_seperable (generic function with 1 method)

In [9]:
#B = random_state(d)
B = ComplexF64[0.33019422880897437 + 0.0im 0.12142404118439183 - 0.04105450879395895im 0.12296054084408255 - 0.12584909898105995im -0.12441289134426448 - 0.1495779235525493im; 0.12142404118439183 + 0.04105450879395895im 0.15182422223904188 + 0.0im -0.004180440890841541 - 0.09431487248574094im -0.05070634251235821 - 0.12412871947039963im; 0.12296054084408255 + 0.12584909898105995im -0.004180440890841541 + 0.09431487248574094im 0.3028476881968294 + 0.0im 0.016865987821744106 - 0.10973464093025546im; -0.12441289134426448 + 0.1495779235525493im -0.05070634251235821 + 0.12412871947039963im 0.016865987821744106 + 0.10973464093025546im 0.21513386075515445 + 0.0im]
println(is_ppt(B))
A = kron(B,B)
println(Ket.ketbra([1,1]))
is_seperable(A)

2
false
[1 1; 1 1]
------
4
4
4
4
-------
4
Determined to be separable via sufficiency of the PPT criterion in small dimensions. Reference:
M. Horodecki, P. Horodecki, and R. Horodecki. Separability of mixed states: Necessary and sufficient conditions. Phys. Lett. A, 223:1-8, 1996.



1

In [11]:
Ket.partial_transpose(B,[2])

4×4 Matrix{ComplexF64}:
    0.330194+0.0im          0.121424+0.0410545im  …  -0.00418044-0.0943149im
    0.121424-0.0410545im    0.151824+0.0im            -0.0507063-0.124129im
    0.122961+0.125849im    -0.124413+0.149578im         0.016866+0.109735im
 -0.00418044+0.0943149im  -0.0507063+0.124129im         0.215134+0.0im

In [16]:
# https://physics.stackexchange.com/questions/571365/how-to-find-the-partial-transpose-of-bipartite-states-from-their-matrix-represen
ρ = [[0,0,0,0] [0,1,-1,0] [0,-1,1,0] [0,0,0,0]]

#ρ

4×4 Matrix{Int64}:
 0   0   0  0
 0   1  -1  0
 0  -1   1  0
 0   0   0  0

In [27]:
Ket.partial_trace(ρ,2)

2×2 Matrix{Int64}:
 1  0
 0  1

In [241]:
length([6,7,8,9])

4